In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [2]:
import pickle
import pandas as pd
from datetime import datetime

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/yokks/miniconda3/envs/week4/lib/python3.11/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/yokks/miniconda3/envs/week4/lib/python3.11/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-03.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [7]:
y_pred.std()

5.556602554785001

In [8]:
y_pred.mean()

12.758556818790902

In [9]:
df['ride_id'] = f'{datetime.now().year:04d}/{datetime.now().month:02d}_' + df.index.astype('str')

In [10]:
df.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
count,3.539827e+06,3539827,3539827,3.429669e+06,3.539827e+06,3.429669e+06,3.539827e+06,3.539827e+06,3.539827e+06,3.539827e+06,3.539827e+06,3.539827e+06,3.539827e+06,3.539827e+06,3.429669e+06,3.429669e+06,3.539827e+06
mean,1.707246e+00,2022-03-16 16:37:35.938137856,2022-03-16 16:51:59.895788800,1.387912e+00,5.665108e+00,1.342189e+00,1.175141e+00,1.333845e+01,1.031789e+00,4.936627e-01,2.588406e+00,4.475753e-01,2.973891e-01,1.990878e+01,2.321207e+00,8.129392e-02,1.439929e+01
min,1.000000e+00,2008-12-31 23:19:56,2008-12-31 23:44:06,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-3.000000e+02,-4.500000e+00,-5.000000e-01,-8.432000e+01,-8.300000e+01,-3.000000e-01,-3.028000e+02,-2.500000e+00,-1.250000e+00,1.000000e+00
25%,1.000000e+00,2022-03-08 20:13:02,2022-03-08 20:25:49.500000,1.000000e+00,1.100000e+00,1.000000e+00,1.000000e+00,7.000000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,3.000000e-01,1.185000e+01,2.500000e+00,0.000000e+00,7.283333e+00
50%,2.000000e+00,2022-03-16 18:26:29,2022-03-16 18:42:01,1.000000e+00,1.830000e+00,1.000000e+00,1.000000e+00,9.500000e+00,5.000000e-01,5.000000e-01,2.160000e+00,0.000000e+00,3.000000e-01,1.536000e+01,2.500000e+00,0.000000e+00,1.171667e+01
75%,2.000000e+00,2022-03-24 16:28:01,2022-03-24 16:43:48.500000,1.000000e+00,3.310000e+00,1.000000e+00,1.000000e+00,1.500000e+01,2.500000e+00,5.000000e-01,3.250000e+00,0.000000e+00,3.000000e-01,2.136000e+01,2.500000e+00,0.000000e+00,1.845000e+01
max,6.000000e+00,2022-05-15 20:33:18,2022-05-15 20:38:57,8.000000e+00,2.862598e+05,9.900000e+01,4.000000e+00,7.000000e+02,9.050000e+00,1.710000e+01,4.660000e+02,2.357000e+02,3.000000e-01,8.433600e+02,2.750000e+00,1.250000e+00,6.000000e+01
std,4.681243e-01,NaN,NaN,9.728462e-01,5.748943e+02,5.418762e+00,4.846125e-01,1.143029e+01,1.235186e+00,7.328840e-02,2.825181e+00,1.798000e+00,3.935751e-02,1.465040e+01,6.818401e-01,3.096991e-01,1.011784e+01


In [11]:
df['ride_id']

df_result = pd.DataFrame()

In [12]:
df_result['ride_id'] = df['ride_id']

In [13]:
df_result['y_pred'] = y_pred

In [14]:
output_file="yellow_tripdata_ride_2022-02"
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [15]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3539827 entries, 0 to 3627881
Data columns (total 2 columns):
 #   Column   Dtype  
---  ------   -----  
 0   ride_id  object 
 1   y_pred   float64
dtypes: float64(1), object(1)
memory usage: 81.0+ MB


In [16]:
df_result['ride_id']

0                2023/06_0
1                2023/06_1
2                2023/06_2
3                2023/06_3
4                2023/06_4
                ...       
3627877    2023/06_3627877
3627878    2023/06_3627878
3627879    2023/06_3627879
3627880    2023/06_3627880
3627881    2023/06_3627881
Name: ride_id, Length: 3539827, dtype: object

In [17]:
y_pred.mean()

12.758556818790902